# HUDF F105W Source Extraction and 3-Color Composite (ASTR-19 Final Project)

This notebook performs source extraction and basic photometric analysis on the **Hubble Ultra Deep Field (HUDF)** F105W image, and then creates a 3‑color composite image using the F160W, F125W, and F105W bands.

### Goals
1. Load the HUDF F105W image using **Astropy**.
2. Estimate and subtract the background with **SEP**.
3. Detect sources and count how many are found.
4. Measure fluxes with aperture photometry and histogram their distribution.
5. Compute the **mean, median, and standard deviation** of the fluxes.
6. Identify the **brightest outlier**, its image position, and its distance from the mean in units of σ.
7. Create a **3‑color RGB composite** of the HUDF using:
   - R → F160W
   - G → F125W
   - B → F105W
   and save it as `HUDF_RGB.png`.


In [ ]:
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt

%matplotlib inline

Load the HUDF F105W Image And Background Subtraction

In [ ]:
f105w_file = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"

# Load F105W image
data_f105w = fits.getdata(f105w_file)
data_f105w = data_f105w.astype(np.float32)
data_f105w = data_f105w.byteswap().newbyteorder()

plt.figure(figsize = (6,6))
plt.imshow(data_f105w, cmap = "gray", origin = "lower")
plt.colorbar(label = "Pixel value")
plt.title("HUDF F105W Image")
plt.savefig("udf_f105w_raw.png", dpi = 150, bbox_inches = "tight")
plt.show()

In [ ]:
# Estimate background
bkg_f105w = sep.Background(data_f105w)
bkg_image_f105w = bkg_f105w.back()
data_f105w_sub = data_f105w - bkg_image_f105w

# plot backgorund
plt.figure(figsize=(6,6))
plt.imshow(bkg_image_f105w, cmap = "gray", origin = "lower")
plt.colorbar(label="Background level")
plt.title("HUDF F105W Background Map")
plt.savefig("udf_f105w_background.png", dpi = 150, bbox_inches = "tight")
plt.show()

Source Extraction on F105W

In [ ]:

thresh = 1.5
objects_f105w = sep.extract(data_f105w_sub, thresh)
num_sources = len(objects_f105w)
print("Number of sources detected in HUDF F105W:", num_sources)
objects_f105w[:5]

Plot Detected Sources on F105W


In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots(figsize = (6,6))
ax.imshow(data_f105w_sub, cmap = "gray", origin = "lower", interpolation = "nearest")

for obj in objects_f105w:
    e = Ellipse(xy=(obj['x'], obj['y']), width = 6 * obj['a'], height = 6 * obj['b'], angle = obj['theta'] * 180.0 / np.pi, edgecolor = 'blue', facecolor = 'none', linewidth = 0.8)
    ax.add_patch(e)
ax.set_title("HUDF F105W: Detected Sources")
ax.set_xlabel("X pixel")
ax.set_ylabel("Y pixel")
plt.savefig("udf_f105w_sources.png", dpi = 150, bbox_inches = "tight")
plt.show()

Aperture Photometry on F105W Sources

In [ ]:

r_ap = 3.0
flux_f105w, fluxerr_f105w, flag_f105w = sep.sum_circle(data_f105w_sub, objects_f105w['x'], objects_f105w['y'], r = r_ap)
print("Number of flux measurements is:", len(flux_f105w))

Histogram of Source Fluxes


In [ ]:
plt.figure(figsize=(7,5))
plt.hist(flux_f105w, bins = 50, histtype = 'step')
plt.xlabel("Flux (F105W, arbitrary units)")
plt.ylabel("Number of Sources")
plt.title("HUDF F105W Source Flux Distribution")
plt.savefig("udf_f105w_flux_histogram.png", dpi = 150, bbox_inches = "tight")
plt.show()

Mean, Median, Standard Deviation, and Brightest Outlier

In [ ]:
mean_flux = np.mean(flux_f105w)
median_flux = np.median(flux_f105w)
std_flux = np.std(flux_f105w)
print("Mean flux (F105W):", mean_flux)
print("Median flux (F105W):", median_flux)
print("Standard deviation of flux (F105W):", std_flux)

max_flux = np.max(flux_f105w)
max_index = np.argmax(flux_f105w)
max_x = objects_f105w['x'][max_index]
max_y = objects_f105w['y'][max_index]

z_max = (max_flux - mean_flux) / std_flux if std_flux > 0 else np.nan
print("\nBrightest source flux:", max_flux)
print("Location of brightest source (x, y):", (max_x, max_y))
print("Brightest source is {:.2f} standard deviations above the mean.".format(z_max))

3-Color False-Color Image of the HUDF

In [ ]:
f160w_file = "hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits"
f125w_file = "hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits"
data_f160w = fits.getdata(f160w_file).astype(np.float32)
data_f125w = fits.getdata(f125w_file).astype(np.float32)

# adding helpers to help with the work flow
def normalize_image(img, p_low = 1, p_high = 99):
    lo = np.percentile(img, p_low)
    hi = np.percentile(img, p_high)
    img_clip = np.clip(img, lo, hi)
    if hi != lo:
        norm = (img_clip - lo) / (hi - lo)
    else:
        img_clip * 0
    return norm

# Normalize each band
R = normalize_image(data_f160w)
G = normalize_image(data_f125w)
B = normalize_image(data_f105w)

# Stack into RGB image
rgb = np.dstack([R, G, B])

# Plot the image
plt.figure(figsize = (6,6))
plt.imshow(rgb, origin = "lower")
plt.title("HUDF 3-Color Composite (R=F160W, G=F125W, B=F105W)")
plt.axis("off")
plt.savefig("HUDF_RGB.png", dpi = 200, bbox_inches = "tight")
plt.show()